In [ ]:
import time
import argparse
import numpy as np
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from math import ceil

In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.optim.lr_scheduler import StepLR

In [ ]:
from utils import load_file, preprocessing, get_vocab, load_embeddings, create_gows, accuracy, generate_batches, AverageMeter
from models import MPAD

In [ ]:
class Parser:
    path_to_dataset='../../../MS_shayan/data_for_mpad.txt'
    path_to_embeddings = '../../../MS_shayan/Github_MedicalWordEmbeddings/BioWordVec_PubMed_MIMICIII_d200.vec.bin'
    no_cuda=False
    epochs=200
    lr=0.001
    hidden=64
    penultimate=64
    message_passing_layers=2
    window_size=10
    directed=True
    use_master_node=True
    normalize=True
    dropout=0.5
    batch_size=128
    patience=20

args = Parser()

In [ ]:
# args = parser.parse_args()
args.cuda = not args.no_cuda and torch.cuda.is_available()

In [ ]:
# Read data
docs, class_labels = load_file(args.path_to_dataset)
print(len(docs), len(class_labels))
print(docs[:2], class_labels[:2])

In [ ]:
docs = preprocessing(docs)
print(len(docs))

In [ ]:
enc = LabelEncoder()
class_labels = enc.fit_transform(class_labels)

In [ ]:
nclass = np.unique(class_labels).size
y = list()
for i in range(len(class_labels)):
    t = np.zeros(1)
    t[0] = class_labels[i]
    y.append(t)

In [ ]:
from sklearn.metrics import roc_curve, confusion_matrix , auc, precision_recall_curve, average_precision_score
from sklearn import metrics
%matplotlib inline
import matplotlib.pyplot as plt
def print_metrics(y_true, y_pred):
    false_positive_rate, recall, thresholds = roc_curve(y_true, y_pred)
    roc_auc = auc(false_positive_rate, recall)
    auprc = average_precision_score(y_true, y_pred)
    print('AUC: ',roc_auc, "AUPRC: ", auprc)


    fpr, tpr, thresholds = metrics.roc_curve(y_true, y_pred)

    # calculate the g-mean for each threshold
    gmeans = np.sqrt(tpr * (1-fpr))
    ix = np.argmax(gmeans)
    print('Best Threshold=%f, G-Mean=%.3f' % (thresholds[ix], gmeans[ix]))


    tn, fp, fn, tp = confusion_matrix(y_true, y_pred > thresholds[ix]).ravel()
    print('TN: ', tn, ", FP: ",fp, ", FN:", fn, ", TP:", tp)
    print("==> Sensitivity (Recall, TPR): %.3f"%(tp/(tp+fn)))
    print("==> Specifity: %.3f"%(tn/(tn+fp)))
    print("==> Positive Predictive Value (PPV) (Precision): %.3f"%(tp / (tp + fp)))
    print("==> Negative Predictive Value (NPV): %.3f"%(tn / (tn + fn)))
    print("==> Accuracy: %.3f"%((tp+tn)/(tn+ fp+ fn+tp)))
    print("==> F1 score: %.3f"%((2*tp)/(2*tp + fp + fn)))

    ns_probs = [0 for _ in range(len(y_true))]
    
    # calculate roc curves
    ns_fpr, ns_tpr, _ = roc_curve(y_true, ns_probs)
    lr_fpr, lr_tpr, _ = roc_curve(y_true, y_pred)
    # plot the roc curve for the model
    plt.plot(ns_fpr, ns_tpr, linestyle='--', label='No Skill, AUC = %0.2f' % 0.5)
    plt.plot(lr_fpr, lr_tpr, marker='.', label = 'Our model: AUC = %0.2f' % roc_auc)
    # axis labels
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    # show the legend
    plt.legend()
    # show the plot
    plt.show()
    
    lr_precision, lr_recall, _ = precision_recall_curve(y_true, y_pred)
    lr_auc = auc(lr_recall, lr_precision)
    # summarize scores
#     print('Logistic: f1=%.3f auc=%.3f' % (lr_f1, lr_auc))
    # plot the precision-recall curves
    y_true = np.array(y_true)
    no_skill = len(y_true[y_true==1]) / len(y_true)
    plt.plot([0, 1], [no_skill, no_skill], linestyle='--', label='No skill, AUPRC = %0.2f' % no_skill)
    plt.plot(lr_recall, lr_precision, marker='.',label = 'Our model: AUPRC = %0.2f' % auprc)
    # axis labels
    plt.xlabel('Recall')
    plt.ylabel('Precision')
    # show the legend
    plt.legend(loc = 'upper right')
    # show the plot
    plt.show()

In [ ]:
vocab = get_vocab(docs)
embeddings = load_embeddings(args.path_to_embeddings, vocab)
adj, features, _ = create_gows(docs, vocab, args.window_size, args.directed, args.normalize, args.use_master_node)

In [ ]:
len(vocab), embeddings.shape, len(adj),  len(features)

In [ ]:
'check tokens :', vocab.keys()

In [ ]:
# for i in range(len(docs)):
#     print(len(docs[i]))

# already saved, no need retrain

In [ ]:
kf = KFold(n_splits=7, shuffle=True, random_state=0)
it = 0
accs = list()

## for averaging AUC
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)



for train_index, test_index in kf.split(y):
    it += 1

    idx = np.random.permutation(train_index)
    train_index = idx[:int(idx.size*0.9)].tolist()
    val_index = idx[int(idx.size*0.9):].tolist()

    n_train = len(train_index)
    n_val = len(val_index)
    n_test = len(test_index)

    adj_train = [adj[i] for i in train_index]
    features_train = [features[i] for i in train_index]
    y_train = [y[i] for i in train_index]

    adj_val = [adj[i] for i in val_index]
    features_val = [features[i] for i in val_index]
    y_val = [y[i] for i in val_index]

    adj_test = [adj[i] for i in test_index]
    features_test = [features[i] for i in test_index]
    y_test = [y[i] for i in test_index]

    adj_train, features_train, batch_n_graphs_train, y_train = generate_batches(adj_train, features_train, y_train, args.batch_size, args.use_master_node)
    adj_val, features_val, batch_n_graphs_val, y_val = generate_batches(adj_val, features_val, y_val, args.batch_size, args.use_master_node)
    adj_test, features_test, batch_n_graphs_test, y_test = generate_batches(adj_test, features_test, y_test, args.batch_size, args.use_master_node)

    n_train_batches = ceil(n_train/args.batch_size)
    n_val_batches = ceil(n_val/args.batch_size)
    n_test_batches = ceil(n_test/args.batch_size)

    # Model and optimizer
    model = MPAD(embeddings.shape[1], args.message_passing_layers, args.hidden, args.penultimate, nclass, args.dropout, embeddings, args.use_master_node)

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adam(parameters, lr=args.lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

    if args.cuda:
        model.cuda()
        adj_train = [x.cuda() for x in adj_train]
        features_train = [x.cuda() for x in features_train]
        batch_n_graphs_train = [x.cuda() for x in batch_n_graphs_train]
        y_train = [x.cuda() for x in y_train]
        adj_val = [x.cuda() for x in adj_val]
        features_val = [x.cuda() for x in features_val]
        batch_n_graphs_val = [x.cuda() for x in batch_n_graphs_val]
        y_val = [x.cuda() for x in y_val]
        adj_test = [x.cuda() for x in adj_test]
        features_test = [x.cuda() for x in features_test]
        batch_n_graphs_test = [x.cuda() for x in batch_n_graphs_test]
        y_test = [x.cuda() for x in y_test]

    def train(epoch, adj, features, batch_n_graphs, y):
        optimizer.zero_grad()
#         print('features.shape, ', features.shape)
        output = model(features, adj, batch_n_graphs)
#         print(output.shape)
#         input('efefe')
        loss_train = F.cross_entropy(output, y)
        loss_train.backward()
        optimizer.step()
        return output, loss_train

    def test(adj, features, batch_n_graphs, y):
        output = model(features, adj, batch_n_graphs)
        loss_test = F.cross_entropy(output, y)
        return output, loss_test

    best_acc = 0

    for epoch in range(args.epochs):
        scheduler.step()
        
        start = time.time()
        model.train()
        train_loss = AverageMeter()
        train_acc = AverageMeter()

        # Train for one epoch
        for i in range(n_train_batches):
            output, loss = train(epoch, adj_train[i], features_train[i], batch_n_graphs_train[i], y_train[i])
            train_loss.update(loss.item(), output.size(0))
            train_acc.update(accuracy(output.data, y_train[i].data), output.size(0))

        # Evaluate on validation set
        model.eval()
        val_loss = AverageMeter()
        val_acc = AverageMeter()

        for i in range(n_val_batches):
            output, loss = test(adj_val[i], features_val[i], batch_n_graphs_val[i], y_val[i])
            val_loss.update(loss.item(), output.size(0))
            val_acc.update(accuracy(output.data, y_val[i].data), output.size(0))
    
        # Print results
        print("Cross-val iter:", '%02d' % it, "epoch:", '%03d' % (epoch + 1), "train_loss=", "{:.5f}".format(train_loss.avg),
            "train_acc=", "{:.5f}".format(train_acc.avg), "val_loss=", "{:.5f}".format(val_loss.avg),
            "val_acc=", "{:.5f}".format(val_acc.avg), "time=", "{:.5f}".format(time.time() - start))
        
        # Remember best accuracy and save checkpoint
        is_best = val_acc.avg >= best_acc
        best_acc = max(val_acc.avg, best_acc)
        if is_best:
            early_stopping_counter = 0
            '''torch.save({
                'epoch': epoch + 1,
                'state_dict': model.state_dict(),
                'optimizer' : optimizer.state_dict(),
            }, 'model_best_'+str(it)+'.pth.tar')'''
        else:
            early_stopping_counter += 1
            print("EarlyStopping: %i / %i" % (early_stopping_counter, args.patience))
            if early_stopping_counter == args.patience:
                print("EarlyStopping: Stop training")
                break

    print("Optimization finished!")

    # Testing
    test_loss = AverageMeter()
    test_acc = AverageMeter()
    print("Loading checkpoint!")
    checkpoint = torch.load('model_best_'+str(it)+'.pth.tar')
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    
    ## for averaging AUC
    test_out_all = []
    test_true_all = []
    
    for i in range(n_test_batches):
        output, loss = test(adj_test[i], features_test[i], batch_n_graphs_test[i], y_test[i])
        test_loss.update(loss.item(), output.size(0))
        test_acc.update(accuracy(output.data, y_test[i].data), output.size(0))
        # *****
        test_out_all.extend(output.data.cpu()[:,1].tolist())
        test_true_all.extend(y_test[i].data.cpu().tolist())
        
    accs.append(test_acc.avg.cpu().numpy())

    # Print results
    print("test_loss=", "{:.5f}".format(test_loss.avg), "test_acc=", "{:.5f}".format(test_acc.avg))
    print_metrics(test_true_all, test_out_all)
    
    viz_fpr, viz_tpr, thresholds = metrics.roc_curve(test_true_all, test_out_all)
    interp_tpr = np.interp(mean_fpr, viz_fpr, viz_tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    false_positive_rate, recall, thresholds = roc_curve(test_true_all, test_out_all)
    viz_roc_auc = auc(false_positive_rate, recall)
    aucs.append(viz_roc_auc)
    
    
    print()
print("avg_test_acc=", "{:.5f}".format(np.mean(accs)))


In [ ]:
fig, ax = plt.subplots()

ax.plot([0, 1], [0, 1], linestyle="--", lw=2, color="r", label="Chance", alpha=0.8)

mean_tpr = np.mean(tprs, axis=0)
mean_tpr[-1] = 1.0
mean_auc = auc(mean_fpr, mean_tpr)
std_auc = np.std(aucs)
ax.plot(
    mean_fpr,
    mean_tpr,
    color="b",
    label=r"Mean ROC (AUC = %0.2f $\pm$ %0.2f)" % (mean_auc, std_auc),
    lw=2,
    alpha=0.8,
)

std_tpr = np.std(tprs, axis=0)
tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
ax.fill_between(
    mean_fpr,
    tprs_lower,
    tprs_upper,
    color="grey",
    alpha=0.2,
    label=r"$\pm$ 1 std. dev.",
)

ax.set(
    xlim=[-0.05, 1.05],
    ylim=[-0.05, 1.05],
    title="Receiver operating characteristic example",
)
ax.legend(loc="lower right")
plt.show()

In [ ]:
print("avg_test_acc=", "{:.5f}".format(np.mean(accs)))

In [ ]:
input()

# Save mpad embeddings

In [ ]:
del MPAD
from models import MPAD  

kf = KFold(n_splits=2, shuffle=False)
it = 0
accs = list()

## for averaging AUC
tprs = []
aucs = []
mean_fpr = np.linspace(0, 1, 100)

# Run ONly once to get all embeddgins 
run_only_once = 1
# Set batch to all data
args.batch_size = 299


for train_index, test_index in kf.split(y):
#     it += 1
    all_index = np.array(range(0,299))
    
#     idx = np.random.permutation(train_index)
#     train_index = idx[:int(idx.size*0.9)].tolist()
#     val_index = idx[int(idx.size*0.9):].tolist()

#     n_train = len(train_index)
#     n_val = len(val_index)
    n_test = len(all_index)

#     adj_train = [adj[i] for i in train_index]
#     features_train = [features[i] for i in train_index]
#     y_train = [y[i] for i in train_index]

#     adj_val = [adj[i] for i in val_index]
#     features_val = [features[i] for i in val_index]
#     y_val = [y[i] for i in val_index]

    adj_test = [adj[i] for i in all_index]
    features_test = [features[i] for i in all_index]
    y_test = [y[i] for i in all_index]

#     adj_train, features_train, batch_n_graphs_train, y_train = generate_batches(adj_train, features_train, y_train, args.batch_size, args.use_master_node)
#     adj_val, features_val, batch_n_graphs_val, y_val = generate_batches(adj_val, features_val, y_val, args.batch_size, args.use_master_node)
    adj_test, features_test, batch_n_graphs_test, y_test = generate_batches(adj_test, features_test, y_test, args.batch_size, args.use_master_node)

#     n_train_batches = ceil(n_train/args.batch_size)
#     n_val_batches = ceil(n_val/args.batch_size)
    n_test_batches = ceil(n_test/args.batch_size)

    # Model and optimizer
    model = MPAD(embeddings.shape[1], args.message_passing_layers, args.hidden, args.penultimate, nclass, args.dropout, embeddings, args.use_master_node)

    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = optim.Adam(parameters, lr=args.lr)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=50, gamma=0.5)

    if args.cuda:
        model.cuda()
#         adj_train = [x.cuda() for x in adj_train]
#         features_train = [x.cuda() for x in features_train]
#         batch_n_graphs_train = [x.cuda() for x in batch_n_graphs_train]
#         y_train = [x.cuda() for x in y_train]
#         adj_val = [x.cuda() for x in adj_val]
#         features_val = [x.cuda() for x in features_val]
#         batch_n_graphs_val = [x.cuda() for x in batch_n_graphs_val]
#         y_val = [x.cuda() for x in y_val]
        adj_test = [x.cuda() for x in adj_test]
        features_test = [x.cuda() for x in features_test]
        batch_n_graphs_test = [x.cuda() for x in batch_n_graphs_test]
        y_test = [x.cuda() for x in y_test]

#     def train(epoch, adj, features, batch_n_graphs, y):
#         optimizer.zero_grad()
#         print('features.shape, ', features.shape)
#         output = model(features, adj, batch_n_graphs)
#         print(output.shape)
#         input('efefe')
#         loss_train = F.cross_entropy(output, y)
#         loss_train.backward()
#         optimizer.step()
#         return output, loss_train

    def test(adj, features, batch_n_graphs, y):
        output = model(features, adj, batch_n_graphs)
        loss_test = F.cross_entropy(output, y)
        return output, loss_test

#     best_acc = 0

#     for epoch in range(args.epochs):
#         scheduler.step()
        
#         start = time.time()
#         model.train()
#         train_loss = AverageMeter()
#         train_acc = AverageMeter()

#         # Train for one epoch
#         for i in range(n_train_batches):
#             output, loss = train(epoch, adj_train[i], features_train[i], batch_n_graphs_train[i], y_train[i])
#             train_loss.update(loss.item(), output.size(0))
#             train_acc.update(accuracy(output.data, y_train[i].data), output.size(0))

#         # Evaluate on validation set
#         model.eval()
#         val_loss = AverageMeter()
#         val_acc = AverageMeter()

#         for i in range(n_val_batches):
#             output, loss = test(adj_val[i], features_val[i], batch_n_graphs_val[i], y_val[i])
#             val_loss.update(loss.item(), output.size(0))
#             val_acc.update(accuracy(output.data, y_val[i].data), output.size(0))
    
#         # Print results
#         print("Cross-val iter:", '%02d' % it, "epoch:", '%03d' % (epoch + 1), "train_loss=", "{:.5f}".format(train_loss.avg),
#             "train_acc=", "{:.5f}".format(train_acc.avg), "val_loss=", "{:.5f}".format(val_loss.avg),
#             "val_acc=", "{:.5f}".format(val_acc.avg), "time=", "{:.5f}".format(time.time() - start))
        
#         # Remember best accuracy and save checkpoint
#         is_best = val_acc.avg >= best_acc
#         best_acc = max(val_acc.avg, best_acc)
#         if is_best:
#             early_stopping_counter = 0
#             torch.save({
#                 'epoch': epoch + 1,
#                 'state_dict': model.state_dict(),
#                 'optimizer' : optimizer.state_dict(),
#             }, 'model_best.pth.tar')
#         else:
#             early_stopping_counter += 1
#             print("EarlyStopping: %i / %i" % (early_stopping_counter, args.patience))
#             if early_stopping_counter == args.patience:
#                 print("EarlyStopping: Stop training")
#                 break

#     print("Optimization finished!")

    # Testing
    test_loss = AverageMeter()
    test_acc = AverageMeter()
    print("Loading checkpoint!")
    checkpoint = torch.load('model_best_3.pth.tar')
    epoch = checkpoint['epoch']
    model.load_state_dict(checkpoint['state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer'])
    
    ## for averaging AUC
    test_out_all = []
    test_true_all = []
    
    for i in range(n_test_batches):
        output, loss = test(adj_test[i], features_test[i], batch_n_graphs_test[i], y_test[i])
        input('stop')
        test_loss.update(loss.item(), output.size(0))
        test_acc.update(accuracy(output.data, y_test[i].data), output.size(0))
        # *****
        test_out_all.extend(output.data.cpu()[:,1].tolist())
        test_true_all.extend(y_test[i].data.cpu().tolist())
        
    accs.append(test_acc.avg.cpu().numpy())

    # Print results
    print("test_loss=", "{:.5f}".format(test_loss.avg), "test_acc=", "{:.5f}".format(test_acc.avg))
    print_metrics(test_true_all, test_out_all)
    
    viz_fpr, viz_tpr, thresholds = metrics.roc_curve(test_true_all, test_out_all)
    interp_tpr = np.interp(mean_fpr, viz_fpr, viz_tpr)
    interp_tpr[0] = 0.0
    tprs.append(interp_tpr)
    false_positive_rate, recall, thresholds = roc_curve(test_true_all, test_out_all)
    viz_roc_auc = auc(false_positive_rate, recall)
    aucs.append(viz_roc_auc)
    
    
    print()
    if run_only_once == 1: 
        break
print("avg_test_acc=", "{:.5f}".format(np.mean(accs)))


In [ ]:
import pickle
with open('../../../MS_shayan/MRN_Time_EDSS.pickle', 'rb') as f:
    result_EDSS = pickle.load(f)
    
MPAD_embedding = np.load('MPAD_embedding.npy')

result_MPAD_embedding = []
for MRN, mpad_embed in zip(list(result_EDSS.keys()), MPAD_embedding):
    result_MPAD_embedding.append((MRN, mpad_embed))
    
with open('result_MPAD_embedding.pickle', 'wb') as f:
    pickle.dump(result_MPAD_embedding, f)